# Requirements and Libraries

In [26]:
!pip install ecdsa
!pip install base58
!pip install psutil


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [91]:
import ecdsa
import binascii
import hashlib
import base58
import time
import os
import psutil
import multiprocessing
from ecdsa import SigningKey, SECP256k1

# First approach

In [106]:
# Function to generate compressed public key from private key
def private_to_public(private_key_hex):
    # Decode the hex private key
    private_key_bytes = binascii.unhexlify(private_key_hex)

    # Create a signing key using the secp256k1 curve
    public_key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)

    return public_key    

def compress_public_key(public_key):
    # Get the corresponding verifying key (public key)
    vk = public_key.verifying_key
    
    # Compressed public key (02 or 03 prefix depending on y-coordinate's parity)
    x = vk.to_string()[:32]  # First 32 bytes (x-coordinate)
    y = vk.to_string()[32:]  # Last 32 bytes (y-coordinate)
    
    # Convert y to an integer to check its parity (odd/even)
    y_int = int(binascii.hexlify(y), 16)
    
    # Compressed key: 02 + x if y is even, 03 + x if y is odd
    if y_int % 2 == 0:
        compressed_public_key = b'\x02' + x
    else:
        compressed_public_key = b'\x03' + x
    
    return binascii.hexlify(compressed_public_key).decode()

# Function to perform Base58Check encoding
def base58check_encode(payload):
    # Perform double SHA-256 hash
    sha256_1 = hashlib.sha256(payload).digest()
    sha256_2 = hashlib.sha256(sha256_1).digest()
    
    # Take the first 4 bytes as the checksum
    checksum = sha256_2[:4]
    
    # Append the checksum to the payload
    payload_with_checksum = payload + checksum
    
    # Perform Base58 encoding
    return base58.b58encode(payload_with_checksum).decode()

# Function to generate Bitcoin-like address from compressed public key
def public_key_to_address(compressed_public_key_hex):
    # Step 1: Perform SHA-256 hashing on the compressed public key
    compressed_public_key_bytes = binascii.unhexlify(compressed_public_key_hex)
    sha256_hash = hashlib.sha256(compressed_public_key_bytes).digest()
    
    # Step 2: Perform RIPEMD-160 hashing on the SHA-256 result
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha256_hash)
    ripemd160_hash = ripemd160.digest()
    
    # Step 3: Add 0x00 in front of the RIPEMD-160 result (for a Bitcoin address)
    versioned_payload = b'\x00' + ripemd160_hash
    
    # Step 4: Base58Check encode the versioned payload
    address = base58check_encode(versioned_payload)
    
    return address

def measure_performance(min_exp, max_exp, target_address):
    # Calculate the min and max values from the exponents
    min_val = 2 ** min_exp
    max_val = 2 ** max_exp - 1
    n = max_val - min_val + 1  # Number of iterations (key space size)

    # Start timer and capture initial CPU/memory usage
    process = psutil.Process()
    initial_cpu = psutil.cpu_percent(interval=None)  # CPU usage at start
    initial_memory = process.memory_info().rss / (1024 * 1024)  # Memory in MB

    # Start timing
    start_time = time.time()

    # Run the function (e.g., generate_keys) with the range
    generate_keys(min_val, max_val, target_address)

    # End timing
    end_time = time.time()
    total_time = end_time - start_time
    average_time_per_iteration = (total_time / n) * 1000  # in milliseconds

    # Capture final CPU and memory usage
    final_cpu = psutil.cpu_percent(interval=None)
    final_memory = process.memory_info().rss / (1024 * 1024)  # Memory in MB

    # Print total time taken and average time per iteration
    print(f'Total time taken: {total_time:.6f} seconds')
    print(f'Average time per iteration: {average_time_per_iteration:.3f} ms')

    # Print CPU and memory usage
    print(f'Initial CPU Usage: {initial_cpu}%')
    print(f'Final CPU Usage: {final_cpu}%')
    print(f'Initial Memory Usage: {initial_memory:.2f} MB')
    print(f'Final Memory Usage: {final_memory:.2f} MB')
    
# Example function to simulate key generation or brute-forcing
def generate_keys(min_val, max_val, target_address):
    
    for i in range(min_val, max_val + 1):  # Iterate through the range of keys
        # Convert the number to hexadecimal and pad it to 64 characters (256 bits)
        private_key_hex = f'{i:064x}'
        
        # Calculate the public key
        public_key = private_to_public(private_key_hex)

        # Calculate the compressed public key
        compressed_public_key = compress_public_key(public_key)
        
        # Calculate the Bitcoin-like address from the compressed public key
        bitcoin_address = public_key_to_address(compressed_public_key)
        
        # Output the private key, compressed public key, and Bitcoin-like address
        if bitcoin_address == target_address:
            print('-' * 80)
            print(f'Private Key (Decimal): {i}')
            print(f'Private Key (Hex): {private_key_hex}')
            print(f'Compressed Public Key: {compressed_public_key}')
            print(f'Bitcoin Address (Base58Check): {bitcoin_address}')
            return  # Exit once the target address is found
    
    # End timer and print if the target address wasn't found in the range
    print('-' * 80)
    print(f'Target address not found in the range from {min_val} to {max_val}.')
        

In [111]:
target_address = '1HsMJxNiV7TLxmoF6uJNkydxPFDog4NQum'
measure_performance(19, 20, target_address)

--------------------------------------------------------------------------------
Private Key (Decimal): 863317
Private Key (Hex): 00000000000000000000000000000000000000000000000000000000000d2c55
Compressed Public Key: 033c4a45cbd643ff97d77f41ea37e843648d50fd894b864b0d52febc62f6454f7c
Bitcoin Address (Base58Check): 1HsMJxNiV7TLxmoF6uJNkydxPFDog4NQum
Total time taken: 24.308894 seconds
Average time per iteration: 0.046 ms
Initial CPU Usage: 23.3%
Final CPU Usage: 27.0%
Initial Memory Usage: 79.67 MB
Final Memory Usage: 79.73 MB


# Measure available system resources

In [112]:

# Estimate memory usage per key conversion (private -> compressed public)
def estimate_memory_per_key():
    # Example private key
    private_key = b'\x01' * 32  # 32-byte key (256-bit)
    
    # Generate compressed public key
    private_key_obj = SigningKey.from_string(private_key, curve=SECP256k1)
    public_key = private_key_obj.get_verifying_key().to_string("compressed")
    
    # Return the size of the private and public key in bytes
    return len(private_key) + len(public_key)

def get_optimal_num_threads():
    # Get the number of logical CPUs on the system
    logical_cpus = os.cpu_count()

    # For a more accurate number, use psutil to detect the number of CPU cores
    cpu_info = psutil.cpu_count(logical=False)  # Physical cores
    
    # Memory constraint: Estimate memory usage per thread
    memory_info = psutil.virtual_memory()
    
    # Conservative estimate: Limit to the number of logical CPUs to avoid overloading
    max_threads = logical_cpus
    
    # Calculate how many threads can be supported by the available memory
    per_thread_memory_usage = estimate_memory_per_key()  # Function should estimate per-thread memory
    max_possible_threads = memory_info.available // (per_thread_memory_usage * 10000)  # Assuming batch of 10k keys
    
    # Final number of threads is the minimum between CPU and memory constraints
    return min(max_threads, max_possible_threads)

def optimal_worker_count():
    num_cores = multiprocessing.cpu_count()
    # Use all available cores, or reduce if IO-bound tasks
    return num_cores

# Calculate max batch size based on available memory and number of threads/processes
def calculate_max_batch_size(num_threads):
    memory_info = psutil.virtual_memory()
    available_memory = memory_info.available  # Available memory in bytes
    
    # Estimate memory usage per key conversion
    memory_per_key = estimate_memory_per_key()
    
    # Allow using only a fraction of the available memory, e.g., 70%
    usable_memory = available_memory * 0.7
    
    # Divide the usable memory by the number of threads to avoid overloading the system
    memory_per_thread = usable_memory / num_threads
    
    # Calculate max batch size per thread
    max_batch_size = memory_per_thread // memory_per_key
    return max_batch_size

In [113]:
print(f'estimated memory per key: {estimate_memory_per_key()} bytes')

num_threads = get_optimal_num_threads()

print(f'max threads: {num_threads}')

print(f"Optimal number of workers: {optimal_worker_count()}")

print(f'max batch size: {calculate_max_batch_size(num_threads)}')


estimated memory per key: 65 bytes
max threads: 12
Optimal number of workers: 12
max batch size: 11622849.0
